In [3]:
import pandas as pd
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.interpolate as interpolate
import mesa_reader as mr
import csv

In [4]:
def radau_equation(t, y, rho_over_rhom):
    r = t
    eta = y[0]
    if r==0.0: return 0.0
    
    detadr = - eta * (eta - 1) + 6 - 6*rho_over_rhom(r) * (eta + 1)
    detadr = detadr / r
    return [detadr]

def get_rho_over_rhom(rho, r):
    rhof = interpolate.interp1d(r, np.log10(rho), kind="linear")
    rs = np.linspace(r[0], r[-1], 100)
    rhom = rs.copy()
        
    rhom[0] = rho[0]
    for i in range(1,len(rhom)):
        intgral, err = integrate.quad(lambda x: x*x*10**rhof(x), r[0], rs[i])
        rhom[i] = 3/rs[i]**3 * intgral
    rho_over_rhom = 10**rhof(rs)/rhom
    rho_over_rhomf = interpolate.interp1d(rs, rho_over_rhom, kind="linear")

    return rho_over_rhomf

In [12]:
kaps_pri = []
kaps_pri_time =[]
kaps_sec = []
kaps_sec_time =[]
N=300
for i in range(N):
    profile_path1 ='../LOGS1/profile%d.data'%(i+1)
    p = mr.MesaData(profile_path1)
    kaps_pri_time.append(p.header('star_age'))
    rho = 10**p.data('logRho')
    r = 10**p.data('logR')
    r = np.flip(r)
    rho = np.flip(rho)
    rho_over_rhomf = get_rho_over_rhom(rho, r)
    eta_center = 0.0
    teval = np.linspace(0.1, r[-1], 50)
    sol = integrate.solve_ivp(radau_equation, t_span=[r[0], r[-1]], y0=[eta_center], args=(rho_over_rhomf,), method="DOP853", t_eval=teval, rtol=1e-9, atol=1e-9)
    eta2_surf = sol.y[0][-1]
    kaps = (3 - eta2_surf) / (4 + 2*eta2_surf)
    kaps_pri.append(kaps)
for i in range(N):
    profile_path2 ='../LOGS2/profile%d.data'%(i+1)
    p = mr.MesaData(profile_path2)
    kaps_sec_time.append(p.header('star_age'))
    rho = 10**p.data('logRho')
    r = 10**p.data('logR')
    r = np.flip(r)
    rho = np.flip(rho)
    rho_over_rhomf = get_rho_over_rhom(rho, r)
    eta_center = 0.0
    teval = np.linspace(0.1, r[-1], 50)
    sol = integrate.solve_ivp(radau_equation, t_span=[r[0], r[-1]], y0=[eta_center], args=(rho_over_rhomf,), method="DOP853", t_eval=teval, rtol=1e-9, atol=1e-9)
    eta2_surf = sol.y[0][-1]
    kaps = (3 - eta2_surf) / (4 + 2*eta2_surf)
    kaps_sec.append(kaps)

/var/folders/dc/bp8tgwzs5xjchjk8z3tl70vm0000gn/T/ipykernel_18841/2432275402.py:17: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  intgral, err = integrate.quad(lambda x: x*x*10**rhof(x), r[0], rs[i])
/var/folders/dc/bp8tgwzs5xjchjk8z3tl70vm0000gn/T/ipykernel_18841/2432275402.py:17: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  intgral, err = integrate.quad(lambda x: x*x*10**rhof(x), r[0], rs[i])


In [15]:
data1 = np.column_stack((kaps_pri, kaps_pri_time))
np.savetxt('kaps_pri.csv', data1, delimiter=',')
data2 = np.column_stack((kaps_sec, kaps_sec_time))
np.savetxt('kaps_sec.csv', data2, delimiter=',')